In [ ]:
%matplotlib widget

from IPython.display import Markdown
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import numpy as np
from joblib import Parallel, delayed

# Capacitance Calculator
In this script we solve for the capacitance of the Tesla coil top-load, usually a toroidal capacitor that couples to the environment. To this end we write down Gauss' law for the electric field $\vec{E}$ as
$$
\nabla \cdot \vec{E} = \frac{\rho}{\varepsilon_0\varepsilon_r},
$$
where $\rho$ is the charge density. Assuming absence of charges in free space, the euqation above reduces to
$$
\nabla \cdot \vec{E} = 0.
$$
We can then introduce the electric potential $U$ that allows to write the electric field as 
$$
\vec{E} = - \nabla U,
$$
and therefore equation 2 reduces to Laplace equation, namely
$$
\nabla^2 U = \frac{\partial^2U}{\partial x^2} + \frac{\partial^2U}{\partial y^2} + \frac{\partial^2U}{\partial z^2} = 0
$$

## Finite Differences
We now use the "finite differences" method to solve for the laplace equation in three dimensions. To this end, we consider the definition of partial derivatives as
\begin{align}
\frac{\partial U}{\partial x} &= \lim_{h \to 0} \frac{U(x + h, y, z) - U(x, y, z)}{h}, \\
\frac{\partial U}{\partial y} &= \lim_{h \to 0} \frac{U(x, y + h, z) - U(x, y, z)}{h}, \\
\frac{\partial U}{\partial z} &= \lim_{h \to 0} \frac{U(x, y, z + h) - U(x, y, z)}{h}.
\end{align}
Similarly, considering another function $f(x,y,z)$, we can write its partial derivatives as
\begin{align}
\frac{\partial f}{\partial x} &= \lim_{h \to 0} \frac{f(x, y, z) - f(x - h, y, z)}{h}, \\
\frac{\partial f}{\partial y} &= \lim_{h \to 0} \frac{f(x, y, z) - f(x, y - h, z)}{h}, \\
\frac{\partial f}{\partial z} &= \lim_{h \to 0} \frac{f(x, y, z) - f(x, y, z - h)}{h},
\end{align}
where we assume $f$ to be derivable on the entirety of $\mathbb{R}^3$. If we now let $f = \partial_{x,y,z}U$ we obtain the following expressions:
\begin{align}
\frac{\partial}{\partial x}\frac{\partial U}{\partial x} &= \lim_{h \to 0} \frac{U(x + h, y, z) - 2U(x, y, z) + U(x - h, y, z)}{h^2}, \\ 
\frac{\partial}{\partial y}\frac{\partial U}{\partial x} &= \lim_{h \to 0} \frac{U(x, y + h, z) - 2U(x, y, z) + U(x, y - h, z)}{h^2}, \\
\frac{\partial}{\partial z}\frac{\partial U}{\partial x} &= \lim_{h \to 0} \frac{U(x, y, z + h) - 2U(x, y, z) + U(x, y, z - h)}{h^2}.
\end{align}
If we now discretize the domain over which we evaluate $U$ such that $U_{i,j,k} = U(ih, jh, kh)$ and relax the limit requirement on the definition of derivative we obtain
\begin{align} 
\frac{\partial^2U}{\partial x^2} &\approx \frac{1}{h^2}\left(U_{i+1,j,k} - 2U_{i,j,k} + U_{i-1,j,k}\right), \\
\frac{\partial^2U}{\partial y^2} &\approx \frac{1}{h^2}\left(U_{i,j+1,k} - 2U_{i,j,k} + U_{i,j-1,k}\right), \\
\frac{\partial^2U}{\partial z^2} &\approx \frac{1}{h^2}\left(U_{i,j,k+1} - 2U_{i,j,k} + U_{i,j,k-1}\right). \\
\end{align}
Finally, from the expression $\nabla^2U = 0$ we get 
$$
U_{i,j,k} = \frac{1}{6}\left(U_{i+1,j,k} + U_{i-1,j,k} + U_{i,j+1,k} + U_{i,j-1,k} + U_{i,j,k+1} + U_{i,j,k-1}\right).
$$
In other words, each sample $U_{i,j,k}$ is the arithmetic average of its neighboring samples. This solution of the discretized Laplace equation allows for a multitude of solving methods.